# 爬去创可贴海报：https://www.chuangkit.com/
进入每个海报子页面，进行图片和标签爬虫

In [ ]:
from selenium import webdriver
# from selenium.webdriver.support.wait import WebDriverWait
import requests
import time
# browser_main = webdriver.Chrome('./chromedriver')
browser = webdriver.Chrome('./chromedriver')


# browser.set_page_load_timeout(100)
# browser.set_script_timeout(10)

In [ ]:
count = 4187
with open('./poster/data.txt','a') as f:
    for i in range(1,38):
        browser_main.get('https://www.chuangkit.com/sj-pi12-or0-pt0-pn{}.html'.format(str(i)))
        elements = browser_main.find_elements_by_class_name('useTemplate')
        for element in elements:
            new_path = element.get_attribute('href')
            print('new_path',new_path)
            try:
                browser.get(new_path)
            except:
                continue

            browser_pic = browser.find_element_by_class_name('left-thumbnail')
            pic_path = browser_pic.find_element_by_tag_name('img').get_attribute('src')
            
            browser_tags = browser.find_element_by_class_name('right-detail')
            tags = browser_tags.find_elements_by_class_name('template-tags-item')
#             time.sleep(20)
            if pic_path is None:
                continue
            
#             print('pic_path:',pic_path)
            r = requests.get(pic_path)
            with open('./poster/{}.png'.format(str(count)), 'wb') as ff:
                ff.write(r.content)
                f.write('./poster/{}.png'.format(str(count)) + '   ')
                count += 1

            
            for tag in tags:
                txt = tag.find_element_by_tag_name('a')
#                 print(str(txt.text))
                f.write(str(txt.text) + '  ')

            f.write('\n')
            f.flush()
        
            

# 爬虫爬取：http://pngimg.com/
分两个块：第一个块是分析主页面的类别标签分布统计；第二块是进入子类页面，开始模拟右键下载图片操作

In [ ]:
import os
import urllib
import ssl
import json
from selenium import webdriver
import requests
import time
browser = webdriver.Chrome('./chromedriver')

ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:

FullNames = {}
catalogs = browser.find_elements_by_class_name('catalog')
for log in catalogs:
    category = log.find_element_by_class_name('category')
    ParaName = category.find_element_by_tag_name('a').get_attribute('href').split('/')[-2]
    print('ParaNmae:',ParaName)
    sub_category = log.find_element_by_class_name('sub_category')
    a=sub_category.find_elements_by_tag_name('a')
    b=sub_category.find_elements_by_tag_name('span')
    subNames = []
    for i,j in zip(a,b):
        intj = int(j.text)
        texti = i.get_attribute('href').split('/')[-2]
        print('subName:',texti)
        if intj >= 90:
            subNames.append([texti,intj])
    if len(subNames) > 0:
        FullNames[ParaName] = subNames

with open('FullPngNames.json','w') as f:
    json.dump(FullNames, f)

In [ ]:
# browser.set_page_load_timeout(100)
with open('FullPngNames.json','r') as f:
    dicts = json.load(f)
for ParaName in dicts:
    subNames = dicts[ParaName]
    for subName in subNames:
        print('ParaNmae:',ParaName,'subName:',subName[0])
        sub_path = '../pngimg/{}/{}'.format(ParaName,subName[0])
        if not os.path.isdir(sub_path):
            os.makedirs(sub_path)
            try:
                browser.get('http://pngimg.com/imgs/{0}/{1}/'.format(ParaName,subName[0]))
            finally:
                count = 1
                alls = browser.find_elements_by_class_name('png_png')
                for a in alls:
                    pic_path = a.find_element_by_tag_name('img').get_attribute('src')
                    if pic_path is None:
                        continue
                    try:
                        r = requests.get(pic_path)
                        with open('../pngimg/{}/{}/{}.png'.format(ParaName,subName[0],subName[0]+'_'+str(count)), 'wb') as ff:
                            ff.write(r.content)
                            count += 1
                            print('it is processing the %dth' % count)
                    except:
                        continue
        else:
            continue
        

In [ ]:
import numpy as np
from PIL import Image
import os
import cv2

In [ ]:
def DelMask(path):
    img = cv2.imread(path,-1)
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGBA)
    h, w, c = img.shape
    if max(h,w) / min(h,w) > 4:
        return None
    mask = img[:,:,3]
    obj = img[:,:,:3]
    obj[mask == 0] = [255,255,255]
    img = Image.fromarray(obj)
    img.thumbnail((1024,1024))
    return img

In [ ]:
count = 1
for root, dirs, files in os.walk('../imgPNG/ori/'):
    for f in files:
        path = os.path.join(root,f)
        try:
            img = DelMask(path)
            if img is not None:
                new_path = root.replace('/ori/','/1024/')
                if not os.path.isdir(new_path):
                    os.makedirs(new_path)
                img.save(path.replace('/ori/','/1024/').replace('png','jpg'))
                print('processing %d pic...' % count)
                count += 1
        except:
            print(path)
            continue

# 读取明星脸[百度百科](https://baike.baidu.com/starrank?fr=lemmaxianhua)

In [ ]:
import sys
import urllib
import time
from bs4 import BeautifulSoup
from selenium import webdriver
# sys.setdefaultencoding('utf8') # 设置编码
url = 'http://baike.baidu.com/starrank?fr=lemmaxianhua'
driver = webdriver.Chrome('./chromedriver') # 创建一个driver用于打开网页，记得找到brew安装的chromedriver的位置，在创建driver的时候指定这个位置
driver.get(url) # 打开网页
name_counter = 1
page = 0;
while page < 2: # 共50页，这里是手工指定的
    soup = BeautifulSoup(driver.page_source, "html.parser")
    current_names = soup.select('div.ranking-table') # 选择器用ranking-table css class，可以取出包含本周、上周的两个table的div标签
    print('current_names',current_names)
    for current_name_list in current_names:
    # print current_name_list['data-cat']
        if current_name_list['data-cat'] == 'thisWeek': # 这次我只想抓取本周，如果想抓上周，改一下这里为lastWeek即可
            names = current_name_list.select('td.star-name > a') # beautifulsoup选择器语法
            counter = 0;
            for star_name in names:
                counter = counter + 1;
                print (star_name.text) # 明星的名字是a标签里面的文本，虽然a标签下面除了文本还有一个与文本同级别的img标签，但是.text输出的只是文本而已
                name_counter = name_counter + 1;
    driver.find_element_by_xpath("//a[contains(text(),'下一页')]").click() # selenium的xpath用法，找到包含“下一页”的a标签去点击
    page = page + 1
    time.sleep(2) # 睡2秒让网页加载完再去读它的html代码
print (name_counter) # 共爬取得明星的名字数量
driver.quit()